# OpenAI Taxi-v3 with Reinforcement Learning & Q-Learning

### Installing necessary packages and libraries

In [4]:
pip install gymnasium


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 18.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pyglet


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install cmake 'gym[atari]' scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 3.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.9 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 18.2 MB/s eta 0:00:0000:0100:01
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827623 sha256=2c12db577038c76eabdd85b1f73630a0797b5942f579ce7e168b86f721ef5acc
  Stored in directory: /Users/ravneet/Library/Caches/pip/wheels/1c/77/9e/9af5470201a0b0543937933ee99ba884cd237d2faefe8f4d37
Successfully built gym

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

In [8]:
import gymnasium as gym
import numpy as np
import random
from IPython.display import clear_output
from time import sleep

### Function to print and animate Environment frames

In [9]:
def print_frames(frames):
  for i, frame in enumerate(frames):
    clear_output(wait=True)
    print(frame['frame'])
    print(f"Timestep: {i + 1}")
    print(f"State: {frame['state']}")
    print(f"Action: {frame['action']}")
    print(f"Reward: {frame['reward']}")
    sleep(.1)

### Initializing Taxi-v3 Environment for observation

In [10]:
env = gym.make("Taxi-v3", render_mode="ansi")
env.reset()

print(f"State space states: {env.observation_space.n}")
print(f"Action space states: {env.action_space.n}")

# Random action
action = env.action_space.sample()
state, reward, done, info, _ = env.step(action)
print(f"State: {state}")
print(f"Action: {action}")
print(f"Reward: {reward}")

frame = env.render()
print(frame)

State space states: 500
Action space states: 6
State: 204
Action: 5
Reward: -10
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



In [11]:
env.unwrapped.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

### Testing Taxi-v3 Environment with Random Agent

In [12]:
env = gym.make("Taxi-v3", render_mode="ansi")
env.s = 328
epochs = 0
penalties, rewards = 0, 0
frames = []
done = False

env.reset()

while not done:
  action = env.action_space.sample()
  state, reward, done , info, _ = env.step(action)

  if reward == -10:
     penalties += 1
 
  frames.append({
      'frame': env.render(),
      'state': state,
      'action': action,
      'reward': reward
       }
  )

  epochs += 1

print("Random Agent Results:\n")
print(f"Timesteps: {epochs} | Penalties: {penalties}")

Random Agent Results:

Timesteps: 1614 | Penalties: 518


### Random Agent with Animation

In [13]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1614
State: 475
Action: 5
Reward: 20


### Q-Table Initialization

In [14]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])
print("Q-Table created:\n\n", q_table)
print("\nShape: ",q_table.shape)

Q-Table created:

 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]

Shape:  (500, 6)


### Training with Q-table

In [15]:
%%time
alpha = 0.1         # Learning Rate
gamma = 0.6         # Discount Factor - determines the importance of future rewards.
epsilon = 0.1       # Exploration-exploitation trade-off - determines the probability of choosing a random action over the action with the highest Q-value.

total_epochs = []
total_penalties = []

for i in range(1, 100001):
    env.reset()
    epochs, penalties, reward, = 0, 1, 1
    done = False
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, info, _ = env.step(action)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)       # Q-learning formula
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        
        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
    
print("Training completed!\n")

Episode: 100000
Training completed!

CPU times: user 16.7 s, sys: 2.92 s, total: 19.7 s
Wall time: 18.7 s


In [16]:
q_table[328]

array([ -2.40084552,  -2.28795447,  -2.41936909,  -2.36604694,
       -11.04703461, -11.07984506])

### Testing after Training with Q-Table

In [17]:
total_epochs = 0
total_penalties = 0
episodes = 100
frames = []

for _ in range(episodes):
    env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False

    while not done:
        action = np.argmax(q_table[state])
        next_state, reward, done, info,_ = env.step(action)

        if reward == -10:
            penalties += 1

        frames.append({
            'frame': env.render(),
            'state': state,
            'action': action,
            'reward': reward
        })

        state = next_state
        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print("\nResults after Q-Learning:\n")
print(f"Total episodes: {episodes}")
print(f"Total timesteps: {total_epochs}")
print(f"Total penalties: {total_penalties}")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")


Results after Q-Learning:

Total episodes: 100
Total timesteps: 1391
Total penalties: 0
Average timesteps per episode: 13.91
Average penalties per episode: 0.0


### Results

In [18]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1391
State: 479
Action: 5
Reward: 20


After training with Q-Table, the agent is making 0 penalties.